Importing Libraries and Dependencies

In [33]:
# Mohammad Malik & Adair Maynard
# Task 5: MAMI
# CMSC 516: NLP at VCU, Dr. Bridget McInnes

!pip install utils
!pip install setuptools
!pip install numpy==1.19

import os
import io
import utils
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import floor

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import tensorflow as tf
import transformers as trfs
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

# from google.colab import auth
# from googleapiclient.discovery import build
# from io import FileIO
# from googleapiclient.http import MediaIoBaseDownload

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from string import punctuation 
from nltk.stem import WordNetLemmatizer

tf.get_logger().setLevel('ERROR')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |██████████████████████

In [34]:
print(trfs.__version__)
tf.__version__

4.12.5


'2.6.0'

In [35]:
# TODO uncomment cell if using Google Colab
#  Get the files from the google drive
# auth.authenticate_user()
# drive_service = build('drive', 'v3')

# # Get data file
# file_id = '1sgE4tBtZwE-M9Y6rfLu_FlwI6VpnaUo-'  # Training file on the Google Drive
# downloaded = io.FileIO("training.csv", 'w')
# request = drive_service.files().get_media(fileId=file_id)
# downloader = MediaIoBaseDownload(downloaded, request)
# done = False
# while done is False:
#   status, done = downloader.next_chunk()
#   print("Download {}%.".format(int(status.progress() * 100)))

In [36]:
#  Load Tokenizer
# TODO look at the finetuned model because it resulted in a lot of UNKNOWN words for capitals and doesnt know a lot of words
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") #Hate-speech-CNERG/dehatebert-mono-english

In [37]:
# Load dataset
df = pd.read_csv("TRAINING/training.csv", delimiter="\t")

df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription
0,1.jpg,0,0,0,0,0,Milk Milk.zip
1,10.jpg,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,0,0,0,0,MAN SEEKING WOMAN Ignad 18 O
4,10006.jpg,0,0,0,0,0,Me explaining the deep lore of. J.R.R. Tolkein...


In [38]:
# get rid of multi classes and only keep misogyny
X = df.drop(columns=['shaming', 'objectification', 'stereotype', 'violence'])
X

,file_name,misogynous,Text Transcription
0,1.jpg,0,Milk Milk.zip
1,10.jpg,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,MAN SEEKING WOMAN Ignad 18 O
4,10006.jpg,0,Me explaining the deep lore of. J.R.R. Tolkein...
...,...,...,...
9995,15002.jpg,0,WAITING FOR THE END OF THE COVID imgflip.com
9996,15003.jpg,0,SMART WOMEN ARE AROUND imgflip.com
9997,15004.jpg,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com
9998,15005.jpg,0,COOKING FOR MY WIFE imgflip.com


In [39]:
# preprocess text then pass into model

# download NLTK stopwords, punctuation and wordLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# clean/preprocess dataset instances
def clean_data(trans):
  words = stopwords.words('english')
  filtered_words = " ".join([w for w in trans.split() if not w.lower() in words]) # might need to be trans.lower().split()
  url = ''.join(re.sub(r'http\S+', '', filtered_words))
  almost = ''.join(re.sub(r'www\S+', '', url))
  cleaned = ''.join(re.sub(r'(\w+\.\w+)', '', almost))
  punct = ''.join(ch for ch in cleaned if ch not in punctuation)
  wordnet_lemmatizer = WordNetLemmatizer()

  word_tokens = nltk.word_tokenize(punct)
  lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]

  word_joined = " ".join(lemmatized_word)
  
  return word_joined

[nltk_data] Downloading package stopwords to /Users/mani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
# apply preprocessing to dataset
X['Cleaned Text'] = X['Text Transcription'].apply(clean_data)

In [41]:
# create new variable that solely consists of cleaned text and labels
X_bert = pd.DataFrame({
    # 'File_Name':X['file_name'],
    'Misogynous':X['misogynous'],
    # 'Alpha':['a']*X.shape[0],
    'Cleaned_Text':X['Cleaned Text']
})
X_bert

,Misogynous,Cleaned_Text
0,0,Milk
1,1,ROSES RED VIOLETS BLUE SAY YES ILL RAPE
2,0,BREAKING NEWS Russia release photo DONALD TRUM...
3,0,MAN SEEKING WOMAN Ignad 18
4,0,explaining deep lore of R Tolkeins world Arda ...
...,...,...
9995,0,WAITING END COVID
9996,0,SMART WOMEN AROUND
9997,0,GOOD GIRLS BEHIND CORNER
9998,0,COOKING WIFE


In [42]:
# get list of sentences and labels
sentences = X_bert.Cleaned_Text.values
print(sentences.shape)

labels = X_bert.Misogynous.values
print(labels)

(10000,)
[0 1 0 ... 0 0 0]


In [43]:
# split dataset into train and test to be passed into models, 85% and 15%
X_train, X_rem, y_train, y_rem = train_test_split(sentences, labels, train_size=0.85)

In [44]:
# split dataset further into valid and test
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, train_size=0.333)

In [45]:
# testing bert tokenizer
print('orig: ', X_train[1])
print('Tokenized: ', tokenizer.tokenize(X_train[1]))
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X_train[1])))

orig:  INTERNATIONAL WOMENS DAY CONGRATULATIONS BORN TWO X CHROMOSOMES MUST COST LOT WORK
Tokenized:  ['international', 'women', '##s', 'day', 'congratulations', 'born', 'two', 'x', 'chromosomes', 'must', 'cost', 'lot', 'work']
Token IDs:  [2248, 2308, 2015, 2154, 23156, 2141, 2048, 1060, 26874, 2442, 3465, 2843, 2147]


In [46]:
MAX_LENGTH = 300

BATCH_SIZE = 64

In [47]:
def batch_encode(tokenizer, texts, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):

  input_ids_batch = []
  attention_mask = []

  for i in range(0, len(texts), batch_size): 
    batch = texts[i:i+batch_size]
    
    inputs = tokenizer.batch_encode_plus(
      batch,
      max_length=MAX_LENGTH, # set the length of the sequences
      add_special_tokens=True, # add [CLS] and [SEP] tokens
      return_attention_mask=True,
      truncation=True, # yells if not set but padding is done
      return_token_type_ids=False, # not needed for this type of ML task
      pad_to_max_length=True, # add 0 pad tokens to the sequences less than max_length
      return_tensors='tf'
      )
    input_ids_batch.extend(inputs['input_ids'])
    attention_mask.extend(inputs['attention_mask'])
  
  return tf.convert_to_tensor(input_ids_batch), tf.convert_to_tensor(attention_mask)

In [48]:
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

/Users/mani/miniforge3/envs/cloud/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [49]:
X_train_ids = np.asarray(X_train_ids)
X_train_attention = np.asarray(X_train_attention)
X_valid_ids = np.asarray(X_valid_ids)
X_valid_attention = np.asarray(X_valid_attention)
X_test_ids =  np.asarray(X_test_ids)
X_test_attention = np.asarray(X_test_attention)

In [50]:
def create_model(transformer, max_length=MAX_LENGTH):    
    # Define input layers
    
    # This is the input for the tokens themselves(words from the dataset after encoding):
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')

    # attention_mask - is a binary mask which tells BERT which tokens to attend and which not to attend.
    # Encoder will add the 0 tokens to the some sequence which smaller than MAX_SEQUENCE_LENGTH, 
    # and attention_mask, in this case, tells BERT where is the token from the original data and where is 0 pad token:

    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    # Use previous inputs as BERT inputs:
    bert = transformer([input_ids_layer, input_attention_layer])[0]

    # We can also add dropout as regularization technique:
    dropped = tf.keras.layers.Dropout(rate=0.15)(bert)

    # Provide number of classes to the final layer:
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dropped)

    # Final model:
    model = tf.keras.models.Model([input_ids_layer, input_attention_layer], outputs=output)


    opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [51]:
# Load Model

# bert_model = trfs.TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased") #Hate-speech-CNERG/dehatebert-mono-english

from transformers import TFDistilBertModel, DistilBertConfig

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2, output_hidden_states=False)
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [52]:
model = create_model(bert_model, MAX_LENGTH)

In [53]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_attention (InputLayer)    [(None, 300)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    input_ids[0][0]                  
                                                                 input_attention[0][0]            
__________________________________________________________________________________________________
dropout_39 (Dropout)            (None, 300, 768)     0           tf_distil_bert_model_1[0][0

In [23]:
history = model.fit(
    x=[X_train_ids, X_train_attention],
    y=y_train,
    epochs=2,
    batch_size=BATCH_SIZE,
    validation_data=([X_valid_ids, X_valid_attention], y_valid)
)

Epoch 1/2
  5/133 [>.............................] - ETA: 59:50 - loss: 0.6911 - accuracy: 0.5515

In [ ]:
yhat_probs = model.predict([X_test_ids, X_test_attention], verbose=0)

In [ ]:
yhat_classes = (model.predict([X_test_ids, X_test_attention]) > 0.5).astype("int32")
print(yhat_classes)

[[[1]
  [1]
  [1]
  ...
  [1]
  [1]
  [1]]

 [[1]
  [1]
  [1]
  ...
  [1]
  [1]
  [1]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[1]
  [1]
  [1]
  ...
  [1]
  [1]
  [1]]]


In [ ]:
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

In [ ]:
#Performance Metrics: Accuracy, precision, recall, F1, cohen's kappa, AUROC, and confusion matrix

# predict probabilities for test set
yhat_probs = model.predict([X_test_ids, X_test_attention], verbose=0)
# predict classes for test set
# yhat_classes = model.predict_classes(X_test, verbose=0) #AttributeError: 'Sequential' object has no attribute 'predict_classes'
yhat_classes = (model.predict([X_test_ids, X_test_attention]) > 0.5).astype("int32")

# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# tp = true positive, tn = true negative, fp = false positive, fn = false negative, p = positive, n = negative
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

# cohen's kappa
kappa = cohen_kappa_score(y_test, yhat_classes)

print('Cohens kappa: %f' % kappa)
# AUROC
auc = roc_auc_score(y_test, yhat_probs)
print('AUROC: %f' % auc)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print('Confusion matrix: ', matrix)

Accuracy: 0.784216
Precision: 0.788136
Recall: 0.762295
F1 score: 0.775000
Cohens kappa: 0.567817
AUROC: 0.861499
Confusion matrix:  [[413 100]
 [116 372]]


In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [29]:
# instead of doing it that way let's also see the lstm
def use_lstm():
  embedding_vector_length = 32
  model = Sequential()
  model.add(Embedding(5000, embedding_vector_length, input_length=MAX_LENGTH))
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(100))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [ ]:
model = use_lstm()

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 32)           9600      
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 32)           3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 150, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 66,005
Trainable params: 66,005
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history2 = model.fit([X_train_ids, X_train_attention], y_train, epochs=3, batch_size=64) #may consider changing # of epochs for speed

Epoch 1/3


InvalidArgumentError:  indices[58,1] = 2047 is not in [0, 300)
	 [[node sequential/embedding/embedding_lookup (defined at var/folders/5p/hmf55qt95d13rj6fqf3q64th0000gn/T/ipykernel_87743/2610551650.py:1) ]] [Op:__inference_train_function_102775]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/101505 (defined at Users/mani/miniforge3/envs/cloud/lib/python3.9/contextlib.py:119)

Function call stack:
train_function


In [ ]:
yhat_probs = model.predict([X_test_ids, X_test_attention], verbose=0)

In [ ]:
yhat_classes = (model.predict([X_test_ids, X_test_attention]) > 0.5).astype("int32")

In [ ]:
print(yhat_classes)

In [ ]:
yhat_classes = model.predict_classes([X_test_ids, X_test_attention], verbose=0) #AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

In [ ]:
#Performance Metrics: Accuracy, precision, recall, F1, cohen's kappa, AUROC, and confusion matrix

# predict probabilities for test set
yhat_probs = model.predict([X_test_ids, X_test_attention], verbose=0)
# predict classes for test set
# yhat_classes = model.predict_classes(X_test, verbose=0) #AttributeError: 'Sequential' object has no attribute 'predict_classes'
yhat_classes = (model.predict([X_test_ids, X_test_attention]) > 0.5).astype("int32")

# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# tp = true positive, tn = true negative, fp = false positive, fn = false negative, p = positive, n = negative
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

# cohen's kappa
kappa = cohen_kappa_score(y_test, yhat_classes)

print('Cohens kappa: %f' % kappa)
# AUROC
auc = roc_auc_score(y_test, yhat_probs)
print('AUROC: %f' % auc)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print('Confusion matrix: ', matrix)

In [ ]:
history_dict = history2.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')